In [2]:
import azureml.core
from azureml.core import Workspace

# print core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.55


In [4]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')

MLtest	eastus	rithinsample	eastus


In [5]:
experiment_name = 'heartathon-demo'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

In [6]:
import os
os.getcwd()

'/Users/Rithin/Desktop/Projects/paragraph-similarity/experiments/notebooks'

In [7]:
data_folder = os.path.join(os.getcwd(), '../data/')
os.makedirs(data_folder, exist_ok=True)

ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

ds.upload(src_dir=data_folder, target_path='dataset', overwrite=True, show_progress=True)

AzureBlob mltest3468790006 azureml-blobstore-94feec12-8b45-4d8e-ade9-6942e29d1cf2
Uploading an estimated of 2 files
Uploading /Users/Rithin/Desktop/Projects/paragraph-similarity/experiments/notebooks/../data/data.csv
Uploading /Users/Rithin/Desktop/Projects/paragraph-similarity/experiments/notebooks/../data/data.pkl
Uploaded /Users/Rithin/Desktop/Projects/paragraph-similarity/experiments/notebooks/../data/data.pkl, 1 files out of an estimated total of 2
Uploaded /Users/Rithin/Desktop/Projects/paragraph-similarity/experiments/notebooks/../data/data.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_b753b3655e4f45639bc2c904c83149a0

In [8]:
import os
script_folder = os.path.join(os.getcwd(), "training_scripts")
os.makedirs(script_folder, exist_ok=True)


In [18]:
import shutil
shutil.copy('../main.py', script_folder)

'/Users/Rithin/Desktop/Projects/paragraph-similarity/experiments/notebooks/training_scripts/main.py'

In [19]:
from azureml.core.compute import AmlCompute
print(ds.path('dataset').as_mount())
print(AmlCompute.supported_vmsizes(workspace=ws))

$AZUREML_DATAREFERENCE_3f2a687888054b9fadbb9b3dcfce49c0
[{'name': 'Standard_DS1_v2', 'vCPUs': 1, 'memoryGB': 3.5}, {'name': 'Standard_DS2_v2', 'vCPUs': 2, 'memoryGB': 7.0}, {'name': 'Standard_DS3_v2', 'vCPUs': 4, 'memoryGB': 14.0}, {'name': 'Standard_DS4_v2', 'vCPUs': 8, 'memoryGB': 28.0}, {'name': 'Standard_DS5_v2', 'vCPUs': 16, 'memoryGB': 56.0}, {'name': 'Standard_DS11_v2', 'vCPUs': 2, 'memoryGB': 14.0}, {'name': 'Standard_DS12_v2', 'vCPUs': 4, 'memoryGB': 28.0}, {'name': 'Standard_DS13_v2', 'vCPUs': 8, 'memoryGB': 56.0}, {'name': 'Standard_DS14_v2', 'vCPUs': 16, 'memoryGB': 112.0}, {'name': 'Standard_DS15_v2', 'vCPUs': 20, 'memoryGB': 140.0}, {'name': 'Standard_D1_v2', 'vCPUs': 1, 'memoryGB': 3.5}, {'name': 'Standard_D2_v2', 'vCPUs': 2, 'memoryGB': 7.0}, {'name': 'Standard_D3_v2', 'vCPUs': 4, 'memoryGB': 14.0}, {'name': 'Standard_D4_v2', 'vCPUs': 8, 'memoryGB': 28.0}, {'name': 'Standard_D11_v2', 'vCPUs': 2, 'memoryGB': 14.0}, {'name': 'Standard_D12_v2', 'vCPUs': 4, 'memoryGB': 28.0

In [29]:
from azureml.train.dnn import PyTorch

script_params = {
    '--dataset_folder': ds.path('dataset').as_mount(),
    '--size':10000
}

pt_est = PyTorch(source_directory=script_folder,
                 script_params=script_params,
                 compute_target='amlcompute',
                 vm_size='Standard_NC6',
                 entry_script='main.py',
                 use_gpu=True,
                pip_packages=["flair"])

WARNING - framework_version is not specified, defaulting to version 1.1.


In [30]:
run = exp.submit(config=pt_est)

In [31]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
heartathon-demo,heartathon-demo_1566092282_036a9ff8,azureml.scriptrun,Starting,Link to Azure Portal,Link to Documentation
